In [168]:
import requests
import pandas as pd
from datetime import date

# https://frost.met.no client ID
client_id = ''

In [169]:
# Bergen (Florida):   SN50540
# Oslo (Blindern):    SN18700
# Trondheim (Lade):   SN68050
# Reed (E39):         SN58060
# Sandane             SN58070
# Grimstad (Landvik): SN38140

endpoint = 'https://frost.met.no/observations/v0.jsonld'
parameters = {
    'sources': 'SN50540, SN18700, SN68050, SN58060, SN58070, SN38140',
    # 'elements': 'sum(precipitation_amount P1M),
    'elements': 'sum(precipitation_amount P1D)',
    'referencetime': '2022-01-01/'+date.today().strftime("%Y-%m-%d"),
    'timeoffsets': 'default'
}
r = requests.get(endpoint, parameters, auth=(client_id,''))
json = r.json()

In [170]:
if r.status_code == 200:
    data = json['data']
    print('Data retrieved from frost.met.no!')
else:
    print('Error! Returned status code %s' % r.status_code)
    print('Message: %s' % json['error']['message'])
    print('Reason: %s' % json['error']['reason'])

Data retrieved from frost.met.no!


In [171]:
df = pd.DataFrame()
for i in range(len(data)):
    row = pd.DataFrame(data[i]['observations'])
    row['referenceTime'] = data[i]['referenceTime']
    row['sourceId'] = data[i]['sourceId']
    # df = df.append(row)
    df = pd.concat([df, row])

df = df.reset_index()
df.head()

,index,elementId,value,unit,timeOffset,timeResolution,timeSeriesId,performanceCategory,exposureCategory,qualityCode,referenceTime,sourceId
0,0,sum(precipitation_amount P1D),0.0,mm,PT6H,P1D,0,C,2,0,2022-01-01T00:00:00.000Z,SN18700:0
1,0,sum(precipitation_amount P1D),7.6,mm,PT6H,P1D,0,C,2,0,2022-01-02T00:00:00.000Z,SN18700:0
2,0,sum(precipitation_amount P1D),0.1,mm,PT6H,P1D,0,C,2,0,2022-01-03T00:00:00.000Z,SN18700:0
3,0,sum(precipitation_amount P1D),0.6,mm,PT6H,P1D,0,C,2,0,2022-01-04T00:00:00.000Z,SN18700:0
4,0,sum(precipitation_amount P1D),0.0,mm,PT6H,P1D,0,C,2,0,2022-01-05T00:00:00.000Z,SN18700:0


In [172]:
# Clean up columns
columns = ['sourceId','referenceTime','elementId','value','unit','timeOffset','timeResolution']
df = df[columns]
df['referenceTime'] = pd.to_datetime(df['referenceTime'])
df.head()

,sourceId,referenceTime,elementId,value,unit,timeOffset,timeResolution
0,SN18700:0,2022-01-01 00:00:00+00:00,sum(precipitation_amount P1D),0.0,mm,PT6H,P1D
1,SN18700:0,2022-01-02 00:00:00+00:00,sum(precipitation_amount P1D),7.6,mm,PT6H,P1D
2,SN18700:0,2022-01-03 00:00:00+00:00,sum(precipitation_amount P1D),0.1,mm,PT6H,P1D
3,SN18700:0,2022-01-04 00:00:00+00:00,sum(precipitation_amount P1D),0.6,mm,PT6H,P1D
4,SN18700:0,2022-01-05 00:00:00+00:00,sum(precipitation_amount P1D),0.0,mm,PT6H,P1D


In [173]:
places = [
{
    "name": "Bergen",
    "id": "SN50540:0"
},
{
    "name": "Oslo",
    "id": "SN18700:0"
},
{
    "name": "Trondheim",
    "id": "SN68050:0"
},
{
    "name": "Sandane",
    "id": "SN58070:0"
},
{
    "name": "Grimstad",
    "id": "SN38140:0"
},
]
# Calculate accumulated precipitation
for n  in range(len(places)):
    place=places[n]
    rain_acc = df.loc[(df["sourceId"] == place["id"])]['value'].sum()
    places[n]["accumulutive_rain"]=rain_acc


print(f'Rain this year until {date.today().strftime("%Y-%m-%d")}')
print(f'Place:       mm:')
print(f'----------------')
for place in places:
 print(f'{place["name"]:10}   {place["accumulutive_rain"]:.0f}')

Rain this year until 2022-10-28
Place:       mm:
----------------
Bergen       1998
Oslo         524
Trondheim    839
Sandane      1076
Grimstad     726
